# Country Education Summary

This notebook processes student-level PISA data to generate a country-level summary of education indicators.

**Source Data**: `data/student_subset_cleaned.csv`

## Load Data

In [8]:
from pathlib import Path
import pandas as pd
import numpy as np
import os

data_dir = Path("data")

# List files to confirm
print("Files in data directory:")
for f in data_dir.glob("*.csv"):
    print(f.name)

Files in data directory:
country_agg.csv
country_agg_life.csv
country_education_summary.csv
country_facts_ranked.csv
Country_Merged.csv
democracy-index-eiu.csv
opinions.csv
Percentage_GDP_and_Percentage_Gov_budget.csv
Percentage_internet_and_electricity.csv
secondary_enrollment_rate_gross.csv
spiderplot.csv
student_subset.csv
student_subset_cleaned.csv


In [9]:
# Load the main dataset
data_file = data_dir / "student_subset_cleaned.csv"
print(f"\nLoading {data_file}...")
df = pd.read_csv(data_file)
print(f"Loaded {len(df)} rows.")
df.head()


Loading data\student_subset_cleaned.csv...
Loaded 613744 rows.


,CNT,CNTSCHID,CNTSTUID,ST001D01T,AGE,ESCS,HISCED,HISEI,Math,Read,Science,LIFESAT,BELONG,BULLIED,FEELSAFE,STRESAGR,PSYCHSYM,SOCCON,EXPWB,ANXMAT
0,Albania,800282.0,800001.0,10.0,16.00,1.1112,5.0,73.91,223.0351,249.8026,301.2603,NaN,-0.2327,-1.2280,1.1246,NaN,NaN,NaN,NaN,0.5440
1,Albania,800115.0,800002.0,9.0,16.25,-3.0507,2.0,24.16,308.4938,288.8999,303.5314,NaN,0.5031,1.3336,1.1246,NaN,NaN,NaN,NaN,NaN
2,Albania,800242.0,800003.0,9.0,15.75,-0.1867,3.4,NaN,313.7353,311.7785,323.6492,NaN,NaN,NaN,0.8637,NaN,NaN,NaN,NaN,NaN
3,Albania,800245.0,800005.0,8.0,15.83,-3.2198,1.0,14.82,298.7323,300.7753,210.1502,NaN,0.5159,0.9885,-0.7560,NaN,NaN,NaN,NaN,0.5653
4,Albania,800285.0,800006.0,10.0,16.33,-1.0548,3.4,30.11,475.7512,486.6689,466.7572,NaN,0.7639,-1.2280,1.1246,NaN,NaN,NaN,NaN,-0.2704


## Column Mapping

Mapping raw columns to target names:
* `CNT` -> `country`
* `Math` -> `math_score`
* `ESCS` -> `ESCS`
* `HISEI` -> `HISEI`
* `BELONG` -> `sense_of_belonging`
* `FEELSAFE` -> `feeling_safe`
* `BULLIED` -> `bullying`

In [10]:
column_mapping = {
    "CNT": "country",
    "Math": "math_score",
    "ESCS": "ESCS",
    "HISEI": "HISEI",
    "HISCED": "HISCED",
    "BELONG": "sense_of_belonging",
    "FEELSAFE": "feeling_safe",
    "BULLIED": "bullying"
}

df.rename(columns=column_mapping, inplace=True)

# Check columns
df.columns

Index(['country', 'CNTSCHID', 'CNTSTUID', 'ST001D01T', 'AGE', 'ESCS', 'HISCED',
       'HISEI', 'math_score', 'Read', 'Science', 'LIFESAT',
       'sense_of_belonging', 'bullying', 'feeling_safe', 'STRESAGR',
       'PSYCHSYM', 'SOCCON', 'EXPWB', 'ANXMAT'],
      dtype='object')

## Filter to Target Countries

We only want: US, Finland, Cambodia, Singapore, Japan, Brazil.

Checking unique country names in the dataset to perform mapping.

In [11]:
# Check unique countries
unique_countries = df['country'].unique()
print("Unique countries in data:", unique_countries)

Unique countries in data: ['Albania' 'Baku (Azerbaijan)' 'Argentina' 'Australia' 'Austria' 'Belgium'
 'Brazil' 'Brunei Darussalam' 'Bulgaria' 'Cambodia' 'Canada' 'Chile'
 'Chinese Taipei' 'Colombia' 'Costa Rica' 'Croatia' 'Czech Republic'
 'Denmark' 'Dominican Republic' 'El Salvador' 'Estonia' 'Finland' 'France'
 'Georgia' 'Palestinian Authority' 'Germany' 'Greece' 'Guatemala'
 'Hong Kong (China)' 'Hungary' 'Iceland' 'Indonesia' 'Ireland' 'Israel'
 'Italy' 'Kosovo' 'Jamaica' 'Japan' 'Kazakhstan' 'Jordan' 'Korea' 'Latvia'
 'Lithuania' 'Macao (China)' 'Malaysia' 'Malta' 'Mexico' 'Mongolia'
 'Republic of Moldova' 'Montenegro' 'Morocco' 'Netherlands' 'New Zealand'
 'Norway' 'Panama' 'Paraguay' 'Peru' 'Philippines' 'Poland' 'Portugal'
 'Qatar' 'Romania' 'Saudi Arabia' 'Serbia' 'Singapore' 'Slovak Republic'
 'Viet Nam' 'Slovenia' 'Spain' 'Sweden' 'Switzerland' 'Thailand'
 'United Arab Emirates' 'Türkiye' 'Ukrainian regions (18 of 27)'
 'North Macedonia' 'United Kingdom' 'United States' 'Urug

In [12]:
# Map to standard names
target_countries = ["US", "Finland", "Cambodia", "Singapore", "Japan", "Brazil"]

# Based on PISA codes/names usually found:
# USA -> US (or 'United States')
# FIN -> Finland
# KHM -> Cambodia
# SGP -> Singapore
# JPN -> Japan
# BRA -> Brazil

# Adjust mapping based on actual values printed above if needed.
# Assuming standard PISA names from previous view of 'Albania' etc.
country_map = {
    "United States": "US",
    "USA": "US",
    "Finland": "Finland",
    "Cambodia": "Cambodia",
    "Singapore": "Singapore",
    "Japan": "Japan",
    "Brazil": "Brazil"
}

# Filter and map
df['country'] = df['country'].replace(country_map)
df_filtered = df[df['country'].isin(target_countries)].copy()

print("Rows after filtering:", len(df_filtered))
df_filtered['country'].value_counts()

Rows after filtering: 43234


country
Brazil       10798
Finland      10239
Singapore     6606
Japan         5760
Cambodia      5279
US            4552
Name: count, dtype: int64

## Compute Summary

Calculating the mean for each indicator by country.

In [13]:
indicators = ["math_score", "ESCS", "HISEI", "HISCED", "sense_of_belonging", "feeling_safe", "bullying"]

summary = df_filtered.groupby("country", as_index=False)[indicators].mean()

summary

,country,math_score,ESCS,HISEI,HISCED,sense_of_belonging,feeling_safe,bullying
0,Brazil,381.136549,-0.975239,41.582246,4.494706,-0.203479,-0.404302,-0.147675
1,Cambodia,327.409293,-2.066908,30.333361,3.395142,-0.449061,-0.573613,-0.088810
2,Finland,475.323953,0.193580,54.702931,5.810873,0.076425,0.341393,-0.386315
3,Japan,534.929480,-0.023547,53.577216,5.305002,0.242962,NaN,-0.728941
4,Singapore,573.983906,0.290373,62.951588,5.586654,-0.229335,0.170973,-0.206616
5,US,462.809686,0.041981,54.801604,5.340494,-0.263331,-0.201599,-0.294152


## Save CSV

In [14]:
output_path = data_dir / "country_education_summary.csv"
summary.to_csv(output_path, index=False)

print(f"Saved summary to {output_path}")
pd.read_csv(output_path)

Saved summary to data\country_education_summary.csv


,country,math_score,ESCS,HISEI,HISCED,sense_of_belonging,feeling_safe,bullying
0,Brazil,381.136549,-0.975239,41.582246,4.494706,-0.203479,-0.404302,-0.147675
1,Cambodia,327.409293,-2.066908,30.333361,3.395142,-0.449061,-0.573613,-0.088810
2,Finland,475.323953,0.193580,54.702931,5.810873,0.076425,0.341393,-0.386315
3,Japan,534.929480,-0.023547,53.577216,5.305002,0.242962,NaN,-0.728941
4,Singapore,573.983906,0.290373,62.951588,5.586654,-0.229335,0.170973,-0.206616
5,US,462.809686,0.041981,54.801604,5.340494,-0.263331,-0.201599,-0.294152
